In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 忽略繁琐警告
warnings.filterwarnings("ignore")

class Graph_Solver_Capsule:
    def __init__(self, name="Graph_Model", is_directed=False):
        """
        [MCM/ICM Graph Solver V6.2]
        核心功能: 最短路径 (SP), 最小生成树 (MST), 网络流 (Max Flow)
        逻辑防御: 
        1. 自动处理不连通区域 (Infinity)
        2. 严格区分有向/无向图 (Symmetry Check)
        3. 强制验证 MST != Shortest Path
        """
        self.name = name
        self.is_directed = is_directed
        
        # === 核心对象 ===
        # 根据指令2: 注意图的有向或者无向
        if self.is_directed:
            self.G = nx.DiGraph()
            print(f"🔧 初始化: 有向图 (Directed Graph) - 邻接矩阵可能不对称")
        else:
            self.G = nx.Graph()
            print(f"🔧 初始化: 无向图 (Undirected Graph) - 邻接矩阵将强制对称")
            
        # === 结果存储 ===
        self.results = {}
        self.pos_layout = None # 布局缓存
        self.computed_metrics = {}

    # ======================================================
    # Phase 0: V6.2 反向握手 (Reverse Handshake)
    # ======================================================
    def generate_handshake(self, df_dict=None):
        """生成发给 AI 的握手指令"""
        print(f"\n🤝 === 请复制以下内容发送给 AI (Handshake Prompt) ===\n")
        print(f"【系统设定】")
        print(f"我正在使用 `Graph_Solver_Capsule` (V6.2)。当前图设定为: {'有向' if self.is_directed else '无向'}。")
        print(f"环境已加载该类，请直接使用实例 `solver`。")
        
        print("\n【可用接口 (API)】")
        print("- solver.build_from_dataframe(df, source='NodeA', target='NodeB', weight='Cost')")
        print("- solver.build_from_matrix(matrix, node_names=None) # 支持邻接矩阵导入")
        print("- solver.audit() # 必须执行! 检查连通性和孤立点")
        print("- solver.solve_shortest_path(source, target, method='dijkstra')")
        print("- solver.solve_mst() # 最小生成树 (仅限无向图)")
        print("- solver.solve_max_flow(source, sink, capacity_attr='weight')")
        print("- solver.visualize(highlight_path=None) # 自动使用 kamada_kawai_layout")
        print("- solver.export_matrices() # 导出邻接矩阵和距离矩阵")
        
        print("\n【数学逻辑陷阱 (Critical Checks)】")
        print("1. **MST != Shortest Path**: 切勿混淆最小生成树路径与最短路径。")
        print("2. **连通性**: 如果图不连通，不可达节点间的距离必须视为 np.inf。")
        print("3. **三角不等式**: Floyd 算法要求无负权回路。")
        
        if df_dict:
            print("\n【数据情报】")
            for name, df in df_dict.items():
                print(f"Dataset {name}: {list(df.columns)}")

    # ======================================================
    # Phase 1: 数据注入 (I/O Isolation)
    # ======================================================
    def build_from_dataframe(self, df, source_col, target_col, weight_col=None, capacity_col=None):
        """从 DataFrame 构建图 (I/O 隔离: 外部读取，内部构建)"""
        print(f"🏗️ 正在从 DataFrame 构建图...")
        for _, row in df.iterrows():
            u, v = row[source_col], row[target_col]
            attr = {}
            if weight_col: attr['weight'] = float(row[weight_col])
            if capacity_col: attr['capacity'] = float(row[capacity_col])
            self.G.add_edge(u, v, **attr)
        print(f"✅ 图构建完成: {self.G.number_of_nodes()} 节点, {self.G.number_of_edges()} 边")

    def build_from_matrix(self, matrix, node_names=None):
        """[指令3] 矩阵化表达形式导入"""
        print(f"🏗️ 正在从邻接矩阵构建图...")
        rows, cols = matrix.shape
        if rows != cols:
            raise ValueError("邻接矩阵必须是方阵")
            
        if node_names is None:
            node_names = [f"N{i}" for i in range(rows)]
            
        for i in range(rows):
            for j in range(cols):
                w = matrix[i][j]
                # 假设 0 或 inf 代表无边 (视具体数据而定，这里假设非0即有边)
                if w != 0 and w != np.inf:
                    self.G.add_edge(node_names[i], node_names[j], weight=w)
        print(f"✅ 矩阵导入完成。")

    # ======================================================
    # Phase 2: 审计与自检 (Audit)
    # ======================================================
    def audit(self):
        """
        [指令1] 模型中将不连通的边处理为无穷
        同时检查孤立点和连通分量
        """
        print("\n🛡️ === Graph Logic Audit ===")
        
        # 1. 连通性检查
        if self.is_directed:
            n_components = nx.number_weakly_connected_components(self.G)
            is_connected = nx.is_weakly_connected(self.G)
            print(f"ℹ️ 连通性 (Weakly): {'✅ 连通' if is_connected else f'⚠️ 非连通 ({n_components} 分量)'}")
        else:
            n_components = nx.number_connected_components(self.G)
            is_connected = nx.is_connected(self.G)
            print(f"ℹ️ 连通性: {'✅ 连通' if is_connected else f'⚠️ 非连通 ({n_components} 分量)'}")

        # 2. 孤立点检查
        isolates = list(nx.isolates(self.G))
        if isolates:
            print(f"⚠️ 发现 {len(isolates)} 个孤立点: {isolates[:5]}... (建议预处理剔除)")
        
        # 3. 负权边检查 (Dijkstra 陷阱)
        neg_edges = [(u,v) for u,v,d in self.G.edges(data=True) if d.get('weight', 0) < 0]
        if neg_edges:
            print(f"❌ [严重警告] 发现负权边! 严禁使用 Dijkstra，请使用 Bellman-Ford 或 Floyd。")
        else:
            print(f"✅ 无负权边，Dijkstra 安全。")
            
        # 4. 布局预计算 (防止画图成毛线团)
        # 优先使用 kamada_kawai (适合分析网络结构)，太慢则回退到 spring
        if len(self.G) < 500:
            self.pos_layout = nx.kamada_kawai_layout(self.G)
        else:
            self.pos_layout = nx.spring_layout(self.G)

    # ======================================================
    # Phase 3: 求解与模型 (Common Models)
    # ======================================================
    def solve_shortest_path(self, source, target, method='dijkstra'):
        """[指令5] 最短路径模型 (SP)"""
        print(f"\n🚀 求解最短路径 ({source} -> {target})...")
        try:
            path = nx.shortest_path(self.G, source=source, target=target, weight='weight', method=method)
            length = nx.shortest_path_length(self.G, source=source, target=target, weight='weight', method=method)
            
            self.results['SP'] = {'path': path, 'length': length}
            print(f"💎 最短路径长度: {length}")
            print(f"📍 路径: {' -> '.join(map(str, path))}")
            return path, length
        except nx.NetworkXNoPath:
            print(f"❌ [结果] 节点 {source} 与 {target} 不连通。距离 = Inf")
            # [指令1] 将不连通处理为无穷
            return None, np.inf

    def solve_mst(self, algorithm='kruskal'):
        """[指令5] 最小生成树模型 (MST) - 仅限无向图"""
        if self.is_directed:
            print("⚠️ MST 通常定义在无向图中。有向图请考虑最小树形图 (Arborescence)。")
            return None
            
        print(f"\n🚀 求解最小生成树 (MST) - Algorithm: {algorithm}...")
        mst = nx.minimum_spanning_tree(self.G, weight='weight', algorithm=algorithm)
        total_weight = sum(d['weight'] for u, v, d in mst.edges(data=True))
        
        self.results['MST'] = mst
        print(f"💎 MST 总权重: {total_weight}")
        print(f"   包含边数: {mst.number_of_edges()}")
        return mst

    def solve_max_flow(self, source, sink):
        """[指令5] 网络流模型"""
        print(f"\n🚀 求解最大流 ({source} -> {sink})...")
        try:
            flow_value, flow_dict = nx.maximum_flow(self.G, source, sink, capacity='capacity')
            self.results['Flow'] = {'value': flow_value, 'dict': flow_dict}
            print(f"💎 最大流值: {flow_value}")
            return flow_value
        except Exception as e:
            print(f"❌ 求解失败: {e} (检查是否设置了 'capacity' 属性)")
            return None

    def verify_mst_vs_sp(self, source, target):
        """[指令4] 专门验证 MST 路径 != 最短路径"""
        if 'MST' not in self.results:
            print("请先运行 solve_mst()")
            return
            
        print(f"\n🔍 === 逻辑验证: MST vs Shortest Path ({source}->{target}) ===")
        # 1. 计算原图的最短路径
        try:
            sp_len = nx.shortest_path_length(self.G, source, target, weight='weight')
        except:
            sp_len = np.inf
            
        # 2. 计算 MST 上的路径 (MST 是一棵树，两点间路径唯一)
        mst = self.results['MST']
        try:
            mst_path_len = nx.shortest_path_length(mst, source, target, weight='weight')
        except:
            mst_path_len = np.inf
            
        print(f"   Original Graph SP Length: {sp_len}")
        print(f"   MST Path Length         : {mst_path_len}")
        
        if mst_path_len > sp_len:
            print("✅ 验证成功: MST 路径确实大于(或等于)原图最短路径。")
            print("   (解释: MST 保证全网总成本最低，但不保证任意两点间最快。)")
        else:
            print("ℹ️ 特殊情况: 此图中两点间的 MST 路径恰好等于最短路径。")

    # ======================================================
    # Phase 4: 可视化与导出 (Visualization & Export)
    # ======================================================
    def visualize(self, highlight_path=None, title="Graph Visualization"):
        """集成高级布局算法，防止画图混乱"""
        plt.figure(figsize=(10, 8))
        pos = self.pos_layout if self.pos_layout else nx.spring_layout(self.G)
        
        # 绘制底图
        nx.draw_networkx_nodes(self.G, pos, node_size=500, node_color='lightblue')
        nx.draw_networkx_edges(self.G, pos, alpha=0.3, edge_color='gray')
        nx.draw_networkx_labels(self.G, pos)
        
        # 绘制权重标签
        edge_labels = nx.get_edge_attributes(self.G, 'weight')
        # 仅显示部分标签以防重叠
        if len(edge_labels) < 50:
            nx.draw_networkx_edge_labels(self.G, pos, edge_labels=edge_labels)
        
        # 高亮路径 (如 SP)
        if highlight_path:
            path_edges = list(zip(highlight_path, highlight_path[1:]))
            nx.draw_networkx_nodes(self.G, pos, nodelist=highlight_path, node_color='orange')
            nx.draw_networkx_edges(self.G, pos, edgelist=path_edges, edge_color='red', width=2)
            
        # 高亮 MST
        if 'MST' in self.results and not highlight_path:
            mst_edges = list(self.results['MST'].edges())
            nx.draw_networkx_edges(self.G, pos, edgelist=mst_edges, edge_color='green', width=2, style='dashed')
            title += " (Green: MST)"
            
        plt.title(title)
        plt.axis('off')
        plt.show()

    def export_matrices(self):
        """[指令3] 矩阵化表达形式导出"""
        print("\n💾 正在导出矩阵数据...")
        nodes = list(self.G.nodes())
        
        # 1. 邻接矩阵 (Adjacency Matrix)
        adj_df = nx.to_pandas_adjacency(self.G, weight='weight')
        adj_df.to_excel(f"{self.name}_Adjacency.xlsx")
        
        # 2. 距离矩阵 (Distance Matrix) - 包含无穷大处理
        # Floyd-Warshall 计算全源最短路
        dist_dict = dict(nx.all_pairs_dijkstra_path_length(self.G, weight='weight'))
        dist_matrix = pd.DataFrame(index=nodes, columns=nodes)
        
        for u in nodes:
            for v in nodes:
                val = dist_dict.get(u, {}).get(v, np.inf) # [指令1] 不连通处理为无穷
                dist_matrix.loc[u, v] = val
        
        dist_matrix.to_excel(f"{self.name}_Distance.xlsx")
        
        print(f"✅ 已保存: {self.name}_Adjacency.xlsx (图结构)")
        print(f"✅ 已保存: {self.name}_Distance.xlsx (全源最短路, Inf=不可达)")
        
        print("\n📝 LaTeX Adjacency Matrix Snippet (Top 5):")
        print(adj_df.iloc[:5, :5].to_latex(float_format="%.2f"))

    def get_methodology(self):
        """[指令 2 & 5] 方法论生成"""
        type_str = "Directed" if self.is_directed else "Undirected"
        return rf"""
        \subsection{{Graph Theoretic Modeling}}
        We modeled the system as a **{type_str} Graph** $G=(V, E)$, where $V$ represents the set of nodes and $E$ represents the connections. 
        The adjacency matrix $A$ was constructed such that $A_{{ij}} = w_{{ij}}$ if an edge exists, otherwise 0.
        
        To analyze the network properties, we employed the following algorithms:
        \begin{itemize}
            \item \textbf{{Shortest Path Analysis}}: We used **Dijkstra's Algorithm** to determine the optimal route with minimum accumulated weight. Connectivity was rigorously checked, with disconnected paths assigned a distance of $\infty$.
            \item \textbf{{Minimum Spanning Tree (MST)}}: To minimize the total network construction cost, we applied **Kruskal's Algorithm**. We explicitly verified that the path on the MST is generally suboptimal compared to the shortest path in the original graph.
            \item \textbf{{Network Flow}}: The capacity constraints were modeled using the **Max-Flow Min-Cut** theorem to evaluate throughput limits.
        \end{itemize}
        Visualization was performed using the Kamada-Kawai force-directed layout algorithm to reveal topological clusters.
        """

## 第二部分：指挥官自然语言工作流 (Workflow Description)

> **核心定位**：这是一套处理“点线关系”的战略地图。图论建模最大的风险在于“肉眼看不见断路”和“混淆路径定义（如 MST 与 SP）”。此流程强制您在计算前确认图的拓扑性质。

### Phase 0: 启动与定性 (Boot & Definition)
* **动作**：实例化类，决定图的“方向性”。
* **关键决策 (Critical Decision)**：
    * **有向图 (`is_directed=True`)**：关系是单向的（如水流、单行道、关注关系）。
        * *后果*：邻接矩阵 $A_{ij} \neq A_{ji}$，A到B能通不代表B到A能通。
    * **无向图 (`is_directed=False`)**：关系是双向的（如修路、握手、也就是距离）。
        * *后果*：邻接矩阵强制对称。
* **V6.2 执行**：运行 `solver.generate_handshake()`。
* **目的**：让 AI 明确当前是处理“单行道”还是“双行道”，防止后续算法调用错误（例如 MST 算法通常仅用于无向图）。

### Phase 1: 构建与雷达审计 (Build & Audit)
* **动作**：注入数据（DataFrame 或 矩阵），然后立即运行 `solver.audit()`。
* **逻辑检查 (Logic Check)**：
    1.  **连通性 (Connectivity)**：
        * *观察*：audit 输出的“连通分量”是否 $>1$？
        * *判定*：如果是，说明图是断开的。
        * *数学处理*：代码会自动将不可达节点间的距离设为 `np.inf`（无穷大）。你需要在论文中解释这一点：“We define the distance between disconnected components as infinity.”
    2.  **负权边 (Negative Weights)**：
        * *观察*：audit 是否警告发现负权边？
        * *判定*：如果有，**严禁使用 Dijkstra**（会死循环或算错），必须切换指令要求使用 Bellman-Ford。

### Phase 2: 模型选择与逻辑注入 (Model Selection)
* **动作**：根据实际问题选择算法模型。
* **决策树 (Decision Tree)**：
    1.  **“我要最快/最近” (Efficiency)** $\rightarrow$ **Shortest Path (SP)**
        * *算法*：Dijkstra (无负权), Bellman-Ford (有负权)。
        * *场景*：导航、急救反应时间、最少中转。
    2.  **“我要最省钱/连通所有点” (Cost/Infrastructure)** $\rightarrow$ **Minimum Spanning Tree (MST)**
        * *算法*：Kruskal 或 Prim。
        * *场景*：铺设光缆、修路连接所有城市、造价最低。
        * *注意*：仅适用于无向图。
    3.  **“我要最大流量” (Throughput)** $\rightarrow$ **Max Flow**
        * *算法*：Ford-Fulkerson / Edmonds-Karp。
        * *场景*：水管最大输水量、铁路最大运力、网络带宽。

### Phase 3: 陷阱防御与验证 (Defense & Verification)
* **核心动作**：运行 `verify_mst_vs_sp(source, target)`。
* **数学陷阱 (The Trap)**：
    * *误区*：很多人认为“最小生成树”上的路径就是两点间的“最短路径”。**这是错的！**
    * *防御*：此函数会直接对比两个数值。通常 $L_{MST} \ge L_{SP}$。
    * *论文写作*：如果在论文中讨论路网规划，必须明确指出：“虽然 MST 降低了总造价，但在 MST 上行走的效率低于原图的最短路径，这是一种 Trade-off。”

### Phase 4: 交付与可视化 (Delivery)
* **动作**：可视化与导出矩阵。
* **可视化策略**：
    * 代码内置了 `kamada_kawai_layout`。如果发现图画出来是一团乱麻，说明节点间的“弹簧力”没平衡好，这是图论可视化的通病，通常不影响计算结果，但在论文中建议只画关键子图。
* **矩阵导出**：
    * 运行 `export_matrices()`。
    * *检查*：打开生成的 Excel，确认不可达的节点是否真的填入了 `inf`。将生成的 LaTeX 代码直接粘贴到论文附录中。

## 第三部分：指挥官作战地图 (Commander Workflow)

> **版本核心**：这是一套处理“点线关系”的战略地图。此流程经过严格审计，完全对应 V6.2 版 `Graph_Solver_Capsule` 类。通过“控制权反转”与“逻辑防御”，防止在计算前混淆图的拓扑性质（如连通性、方向性）。

### 🔵 Phase 0: 启动与握手 (Boot & Handshake)

* **动作执行者**：指挥官 (你)
* **操作环境**：Jupyter Notebook / Python IDE
* **核心目的**：加载“胶囊”类，注入数据，并生成发给 AI 的“操作说明书”。

请直接复制并运行以下 Python 代码：

```python
# ==========================================
# [步骤 1] 必须先运行之前的 Graph_Solver_Capsule 类定义代码
# (请确保类代码已在上方单元格运行过)
# ==========================================
import pandas as pd
import numpy as np

# [步骤 2] 准备数据 (在此处替换为你真实的 Excel/CSV 读取逻辑)
# 示例：模拟读取边表 (Source, Target, Weight, Capacity)
# ⚠️ 铁律: 不要在类内部 read_csv，必须在这里读好
df_data = pd.DataFrame({
    'Source': ['A', 'A', 'B', 'C', 'D', 'E', 'F'],
    'Target': ['B', 'C', 'D', 'D', 'E', 'A', 'G'], 
    'Weight': [10,  15,  12,  10,   5,   20,  50],  # 距离/成本
    'Cap':    [100, 50,  80,  40,   100, 20,  10]   # 容量(网络流用)
})

# [步骤 3] 实例化胶囊 (关键决策 Point!)
# is_directed=True  -> 单行道、水流、关注关系 (邻接矩阵不对称)
# is_directed=False -> 双向修路、通信、物理距离 (邻接矩阵对称)
solver = Graph_Solver_Capsule(name="My_Graph_Model", is_directed=False)

# [步骤 4] 发射握手信号
# 这会打印一段 "System Prompt"，请复制它。
solver.generate_handshake(df_dict={'Edge_Data': df_data})
```

> **🔴 关键动作**：代码运行后，屏幕会打印 `🤝 === 请复制以下内容发送给 AI ===`。请全文复制那段打印出的文本，发送给 AI。

---

### 🟡 Phase 1: 数据注入与审计 (Build & Audit)

* **动作执行者**：AI (Copilot/ChatGPT)
* **操作方式**：发送 Prompt
* **核心目的**：建立图结构，并利用 `audit()` 进行防御性检查。

请发送以下指令：

> "收到握手信号。
> 请执行以下操作初始化模型：
> 1. 调用 `solver.build_from_dataframe(df_data, source_col='Source', target_col='Target', weight_col='Weight')`。
> 2. **必须立即运行 `solver.audit()` 进行逻辑自检。**
> 3. 根据审计结果，告诉我图是否连通？是否存在负权边？"

**🧐 指挥官自检 (Self-Check)：**
* 如果 AI 回复 “⚠️ 非连通” $\rightarrow$ **记住**：不可达节点间的距离将是 `inf`。
* 如果 AI 回复 “❌ 发现负权边” $\rightarrow$ 下一步**严禁使用 Dijkstra 算法**。

---

### 🟠 Phase 2: 任务下达 (Solve)

* **动作执行者**：AI
* **操作方式**：选择下方一个模板填空发送。

#### 选项 A：我要找最短路径 (导航/时效)
> "请计算从 `[起点]` 到 `[终点]` 的最短路径。
> 1. 运行 `solver.solve_shortest_path(source='...', target='...', method='dijkstra')`。
> 2. 运行 `solver.visualize(highlight_path=...)` 展示结果。"

#### 选项 B：我要修路/铺管网 (最小生成树 MST)
> "请计算最小生成树 (MST) 以最小化总成本。
> 1. 运行 `solver.solve_mst()`。
> 2. **逻辑验证**：运行 `solver.verify_mst_vs_sp(source='...', target='...')`，帮我验证 MST 路径是否长于最短路径。
> 3. 运行 `solver.visualize()` 展示 MST 结构。"

#### 选项 C：我要导出结果写论文 (Delivery)
> "我需要交付模型结果。
> 1. 运行 `solver.export_matrices()` 导出 Excel 和 LaTeX 代码 。
> 2. 打印 `solver.get_methodology()` 获取英文建模描述 。"

---

### 🟢 Phase 3: 结果验收 (Verification)

* **动作执行者**：指挥官 (你)
* **核心目的**：确保结果符合物理和数学逻辑。

1.  **视觉验收**：
    * 查看生成的图片。如果节点挤在一起，指示 AI：“图太乱，请重新运行 `visualize`，布局算法会自动调整。”
    * *(类中已集成 `kamada_kawai` 布局防止画图成毛线团)*。
2.  **无穷大验收**：
    * 打开生成的 `_Distance.xlsx`。
    * 确认不连通的节点对（如 A 到 G）的值是否为 `inf`。
3.  **方法论验收**：
    * 复制 AI 输出的 `get_methodology()` 文本。
    * 确认文中描述的是 "Directed Graph" 还是 "Undirected Graph"，**必须与你 Phase 0 的设置一致**。

`printf()`